In [411]:
import pandas as pd
import pickle
from sklearn.utils import resample
import string
from nltk.stem import WordNetLemmatizer
import spacy
import re
import os
import nltk
import numpy as np
import itertools
from sklearn.utils import resample
from sklearn.metrics import classification_report
lemma = nltk.WordNetLemmatizer()
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import math as mt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import re
import string
import numpy as np
from sklearn import metrics
from scipy.spatial.distance import cdist

In [412]:
data=pd.read_excel('C:\\Users\\gireesh.muppalla\\Downloads\\detail_user_count.xlsx',sheet_name='Sheet1')
data.dropna(subset=['id'],inplace=True)
data1=pd.read_excel('C:\\Users\\gireesh.muppalla\\Desktop\\FInal_post_correction.xlsx',sheet_name='XH_chat_final (003)')
data1.dropna(subset=['id'],inplace=True)

In [413]:
# data2=pd.read_excel('C:\\Users\\gireesh.muppalla\\Downloads\\detail_user_count1.xlsx',sheet_name='with_out_agent_involvement')
# data2.dropna(subset=['id'],inplace=True)
data3=pd.read_excel('C:\\Users\\gireesh.muppalla\\Downloads\\XH Accounts_Region and Division.xlsx',sheet_name='Account Mappings')
data3.dropna(subset=['XH Issue ID'],inplace=True)

data2=pd.read_csv('C:\\Users\\gireesh.muppalla\\Downloads\\agent_info.csv')

In [414]:
data2.head()

,Unnamed: 0,id,if_agent_involved,no_of_agents_involved,users
0,1039,800291,1,4,"['Anabel & Joe Villarreal', 'Automations', 'Fi..."
1,377,827430,1,4,"['LETTICIA LOPEZ', 'Automations', 'Final - Ala..."
2,1522,763700,1,3,"['BEVERLY BROWN', 'Automations', 'Final - Alar..."
3,450,831445,1,3,"['LARRY AKIN', 'Automations', 'Final - Alarm P..."
4,1806,745462,1,3,"['DANIEL MAYE', 'Automations', 'Final - Alarm ..."


In [290]:
#data2=data2[['id','agent_involved','if_bot_involved','num_of_users']].drop_duplicates()

In [415]:
data2.columns

Index(['Unnamed: 0', 'id', 'if_agent_involved', 'no_of_agents_involved',
       'users'],
      dtype='object')

In [416]:
data2.drop('Unnamed: 0',axis=1,inplace=True)

In [418]:
data3.columns=['id','Account ID / User ID','Issue Title','Division','Region']

In [419]:
data3=pd.merge(data3,data2,on='id',how='inner')

In [420]:
#data3.head()

In [421]:
names_dic={}
customer={}
name=[]
id=[]
Customer_text=[]
for i in data['id'].unique():
    #print(i)
    users=data[data['id']==i]['name'].unique().tolist()
    names_dic[i]=users
    customer[i]=' '.join([str(i) for i in data[(data['id']==i) & (data['name']==users[0])]['body'].values.tolist()if i is not np.nan])
    customer[i]=users[0]
    id.append(i)
    name.append(users[0])
    txt=data[(data['id']==i) & (data['name']==users[0])]['body'].values.tolist()
    Customer_text.append(' '.join([str(i) for i in txt if i is not np.nan]))

In [422]:
data4=pd.DataFrame({"Customer_text":Customer_text,'id':id,'Customer_name':name})

In [423]:
data4.shape

(2480, 3)

In [424]:
len(data1.id.unique())

2480

In [425]:
data=pd.merge(data1,data4,on='id',how='inner')

In [426]:
data.shape

(2483, 7)

In [427]:
data=pd.merge(data,data3,on='id',how='inner')

In [428]:
data.shape

(2483, 14)

In [430]:
data[data.id.isin([724152,727523,729442,731967,740604,741404,754910])]

,id,Cluster_x,Subcluster,issue,full_chat,Customer_name,Customer_text,Account ID / User ID,Issue Title,Division,Region,if_agent_involved,no_of_agents_involved,users
197,724152,Keypad_Issues,Panic_Button_Feature,Does my security system keypad have some type ...,['Does my security system keypad have some typ...,Mr & Mrs Brian Jackson,Does my security system keypad have some type ...,8220220700466536,Does my security system keypad have some type ...,Central,BIG SOUTH REGION,0,0,"['Mr & Mrs Brian Jackson', 'Automations', 'Fin..."
290,727523,Keypad_Issues,Keypad_pair/reset/troubleshoot,My touchscreen is saying entry delay is in pro...,['My touchscreen is saying entry delay is in p...,megan tillery,My touchscreen is saying entry delay is in pro...,8396500042115604,My touchscreen is saying entry delay is in pro...,Central,BIG SOUTH REGION,0,0,"['megan tillery', 'Automations', 'Final - Alar..."
335,729442,Keypad_Issues,Entry_delay_Alert,My touchscreen is stuck on Entry delay in prog...,['My touchscreen is stuck on Entry delay in pr...,CHARITY BROCK,My touchscreen is stuck on Entry delay in prog...,8396510770042078,My touchscreen is stuck on Entry delay in prog...,Central,BIG SOUTH REGION,0,0,"['CHARITY BROCK', 'Automations', 'Final - Alar..."
395,731967,Keypad_Issues,Entry_delay_Alert,Hi! My display seems to need to be reset. I ca...,['Hi! My display seems to need to be reset. I ...,Diane Eck,Hi! My display seems to need to be reset. I ca...,8155400020581250,Hi! My display seems to need to be reset. I ca...,West,CALIFORNIA MARKET,0,0,"['Diane Eck', 'Automations', 'Final - Alarm Is..."
581,740604,Alarm_Issues,Arm/Disarm/False_Alarm,Can you please explain the &ldquo;stay&rdquo; ...,['Can you please explain the &ldquo;stay&rdquo...,CYNTHIA WALKER,Can you please explain the &ldquo;stay&rdquo; ...,8777701380736444,Can you please explain the Ã¢â‚¬Å“stayÃ¢â‚¬ï¿½...,West,HOUSTON MARKET,0,0,"['CYNTHIA WALKER', 'Automations', 'Final - Ala..."
605,741404,Keypad_Issues,Entry_delay_Alert,Hello. My security system has been froZen all ...,['Hello. My security system has been froZen al...,STEPHEN FERRARO,Hello. My security system has been froZen all ...,8773400150536696,Hello. My security system has been froZen all ...,NED,BOSTON MARKET,0,0,"['STEPHEN FERRARO', 'Automations', 'Final - Al..."
872,754910,Keypad_Issues,Keypad_pair/reset/troubleshoot,How to reset stuck key pad,"['How to reset stuck key pad', 'Xfinity Home G...",DARRIN COLEY,How to reset stuck key pad Issues with Keypad ...,8299400471243076,How to reset stuck key pad,NED,BELTWAY,0,0,"['DARRIN COLEY', 'Automations', 'Final - Alarm..."


In [431]:
#data.head()

In [432]:
# Deriving Bot_Resolved feature
data['If_Bot_resolved']=''
for ind, row in data.iterrows():
    if row['if_agent_involved']==0:
        data.loc[ind,'If_Bot_resolved']=1
    else:
        data.loc[ind,'If_Bot_resolved']=0
        

In [435]:
# for validation
for i in range(len(data)):
    if data['if_agent_involved'][i]==0 and data['If_Bot_resolved'][i]==0:
        print('issue')

# Model building for main labels

In [437]:
wordnet_lemmatizer = WordNetLemmatizer()
porter=PorterStemmer()
stop_words = stopwords.words('english')

def lemmatize(sentence):
    return ' '.join([lemma.lemmatize(word, 'v') for word in sentence.split(" ")])
                    
def clean(line):
    line = re.sub(r'[?,",:,.,/,!]', ' ', line)
    line = re.sub('[$]', 'Replaceddollar ', line)
    line= re.sub("[^a-zA-Z]", ' ', line)
    line=[i.lower() for i in line.split(' ') if i.lower() not in stop_words]  # Removing the stopwords excluding few neg words
    line=[wordnet_lemmatizer.lemmatize(i.lower(),pos="a") for i in line]
    line=[wordnet_lemmatizer.lemmatize(i.lower(),pos="n") for i in line]
    line=[wordnet_lemmatizer.lemmatize(i.lower(),pos="r") for i in line]
    line=[wordnet_lemmatizer.lemmatize(i.lower(),pos="v") for i in line]
    
    line=" ".join([porter.stem(j) for j in line if len(j)>1 and (j.isnumeric() is False)])
    line = re.sub('  +', ' ', line) # Removing the extra spaces
    return line.strip() #Returning the stripped sentence

In [438]:
data.columns

Index(['id', 'Cluster_x', 'Subcluster', 'issue', 'full_chat', 'Customer_name',
       'Customer_text', 'Account ID / User ID', 'Issue Title', 'Division',
       'Region', 'if_agent_involved', 'no_of_agents_involved', 'users',
       'If_Bot_resolved'],
      dtype='object')

In [439]:
data['cleaned_text']=data['issue'].apply(lambda x: clean(x))

In [440]:
X_train,X_test,y_train,y_test=train_test_split(data['cleaned_text'], data['Cluster_x'],random_state=1)

In [441]:
data['Cluster_x'].value_counts()

Keypad_Issues                     881
Password_Reset/Forget_Password    861
Need_Security_Certificate         599
Alarm_Issues                      142
Name: Cluster_x, dtype: int64

In [442]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=3, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
#vectorizer = TfidfVectorizer(ngram_range=(1,2))
# tokenize and build vocab
vectorizer.fit(X_train)

TfidfVectorizer(encoding='latin-1', min_df=3, ngram_range=(1, 2),
                stop_words='english', sublinear_tf=True)

In [443]:
# from imblearn.over_sampling import SMOTE
# oversample = SMOTE(k_neighbors=26, random_state=123)
# X, y = oversample.fit_resample(vectorizer.transform(X_train), y_train)

In [444]:
from sklearn.svm import LinearSVC
lsvm=LinearSVC()
lsvm.fit(vectorizer.transform(X_train),y_train)
#lsvm.fit(X,y)

LinearSVC()

In [445]:
y_pred=lsvm.predict(vectorizer.transform(X_test))

In [446]:
confusion_matrix(y_test,y_pred)

array([[ 17,   6,   0,   7],
       [  7, 217,   0,   3],
       [  0,   0, 160,   0],
       [  1,   9,   0, 194]], dtype=int64)

In [447]:
print(classification_report(y_test,y_pred))

                                precision    recall  f1-score   support

                  Alarm_Issues       0.68      0.57      0.62        30
                 Keypad_Issues       0.94      0.96      0.95       227
     Need_Security_Certificate       1.00      1.00      1.00       160
Password_Reset/Forget_Password       0.95      0.95      0.95       204

                      accuracy                           0.95       621
                     macro avg       0.89      0.87      0.88       621
                  weighted avg       0.94      0.95      0.95       621



# Model Bulding for Sub-labels

In [451]:
training_data=data[pd.notnull(data['Subcluster'])]
test_data=data[pd.isnull(data['Subcluster'])]

In [452]:
training_data.shape

(1329, 16)

In [453]:
test_data.shape

(1154, 16)

In [454]:
stop_sentences=["Xfinity Home Guide",
                "Bot Started",
                "If you're having an emergency right now",
                "please call 911",
                "Dial 911 to stop a crime",
                "report a fire",
                "or save a life",
                "Thanks for contacting us",
#                 "Confirm your Xfinity Home system is disarmed",
#                 "Working on devices while the system is armed can generate false alarms",
#                 "Remove the Keypad from the wall by sliding upwards until it comes free of the mounting bracket",
#                 "Removing Keypad from the mounting bracket will trigger a Tamper warning on the Touchscreen Controller",
#                 "This will go away once the Keypad is put back on the mounting bracket",
#                 "Remove and replace the batteries",
#                 "Replace the Keypad on the mounting bracket",
#                 "At this time, the Tamper warning disappears",
#                 "Arm the system",
                "Did that resolve your issue",
                "No, connect to a specialist",
                "We will be here to support you if you can chat with us again in the future",
                "Bot Ended",
                "It's been a while since we last heard from you",
                "You can initiate a new chat anytime during our chat hours (7am - 2am EDT)",
                "It seems like you are busy at this time"
               "Please acknowledge my last message","please call us at 1-800-XFINITY (1-800-934-6489)",
                "We will be here to support you if you can chat with us again in the future",
                "visit support.xfinity.com/chat for live agent support at any time",
                "If you need help outside of our chat business hours (7am -2am)",
                "It's been an hour since we last heard from you", "we will save & close this chat",
                "Thank you for choosing XFINITY",
                "It was my pleasure to assist you today",
                "Awesome","I apologize for the wait time in queue",
                "You are most welcome",
                "When you have a moment, please ensure your Emergency Contact information is up to date",
                "You can review and update these from the Xfinity Home mobile app and Subscriber Portal",
                "https://www.xfinity.com/support/home-security/add-change-emergency-contact-mobile-app",
                "https://www.xfinity.com/support/articles/manage-emergency-dispatch-contacts",
                "You have been great",
                "Before we connect you to a specialist",
                "Let me quickly help you in this concern",
                "All of our specialists are currently busy and will chat with you shortly",
                "Please make sure you have push notifications enabled to see your specialist's reply","I apologize for the inconvenience",
                "We value your business as a customer", "Have a great day ahead","Accepted the solution","Good Afternoon","Good Morning",
                "It was a pleasure to help","May I have your complete name","May I have your good name","May I have your full name",
                "Are we good so far","What would you like more information on","I hope you are doing good","Please be assured I will surely help you with this",
               "Keypad codes are used to access the Xfinity Home Security System while the Central Station passcode is a security feature that allows you to identify yourself or someone you trust to the Central Monitoring Station in the event of an alarm",
               "Connect to a specialist","sure","You're welcome","You are welcome","got it",'thanks',"Maintaining these is important in the event that an alarm occurs",
               "Thank you","thanks a lot","I appreciate your efforts and patience","I appreciate your cooperation and patience throughout the chat",
               "Have a great day","Hi","Hello","bye"]
stop_sentences=[i.lower() for i in stop_sentences]

In [455]:
# Taking all chat info

In [456]:
def remove_sentences(x):
    for i in stop_sentences:
        x=x.replace(i,"")
    return x    

In [457]:
training_data['full_chat2']=training_data['Customer_text'].apply(lambda x: remove_sentences(x))
test_data['full_chat2']=test_data['Customer_text'].apply(lambda x: remove_sentences(x))

C:\Users\Gireesh.Muppalla\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Gireesh.Muppalla\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [458]:
training_data['Subcluster']=training_data['Subcluster'].apply(lambda x: x.lower())

C:\Users\Gireesh.Muppalla\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [459]:
training_data['cleaned_full_text']=training_data['full_chat2'].apply(lambda x: clean(x))
test_data['cleaned_full_text']=test_data['full_chat2'].apply(lambda x: clean(x))


C:\Users\Gireesh.Muppalla\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Gireesh.Muppalla\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [460]:
training_data['Subcluster'].value_counts()

security_alarm_certificate           563
reset_master_code                    185
entry_delay_alert                     93
others                                90
keypad_pair/reset/troubleshoot        80
reset_central_station_passcode        80
arm/disarm/false_alarm                72
manage_keypad_codes                   51
low_bat_alert/new_bat_not_working     22
reset_home_app_password               16
keypad_cont_blinking                  15
comm_falure_alert                     14
entry_exit_delay_setting              12
new_keypad_enquiry                    11
panic_button_feature                  11
tamper_alert                           8
reset_secret_question                  6
Name: Subcluster, dtype: int64

In [461]:
X_train,X_test,y_train,y_test=train_test_split(training_data['cleaned_full_text'], training_data['Subcluster'],stratify=training_data['Subcluster'],random_state=5,test_size=0.25)

In [462]:
#a=X_train.append(training_data[training_data.index.isin(train_new_bat)]['cleaned_full_text'])

In [463]:
#y=y_train.append(training_data[training_data.index.isin(train_new_bat)]['Subcluster'])

In [464]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
vectorizer = TfidfVectorizer(sublinear_tf=False, min_df=3, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
#vectorizer = TfidfVectorizer(ngram_range=(1,2))
# tokenize and build vocab
vectorizer.fit(X_train)

TfidfVectorizer(encoding='latin-1', min_df=3, ngram_range=(1, 2),
                stop_words='english')

In [465]:
# Oversampling
from imblearn.over_sampling import SMOTE
oversample = SMOTE(k_neighbors=4, random_state=123)
X, y = oversample.fit_resample(vectorizer.transform(X_train), y_train)

In [466]:
y.value_counts()

comm_falure_alert                    422
entry_exit_delay_setting             422
security_alarm_certificate           422
low_bat_alert/new_bat_not_working    422
reset_master_code                    422
keypad_cont_blinking                 422
keypad_pair/reset/troubleshoot       422
panic_button_feature                 422
new_keypad_enquiry                   422
reset_central_station_passcode       422
arm/disarm/false_alarm               422
others                               422
manage_keypad_codes                  422
reset_home_app_password              422
tamper_alert                         422
entry_delay_alert                    422
reset_secret_question                422
Name: Subcluster, dtype: int64

In [467]:
from sklearn.svm import LinearSVC
lsvc=LinearSVC()
#lsvc.fit(vectorizer.transform(X_train),y_train)
lsvc.fit(X,y)

LinearSVC()

In [468]:
y_pred=lsvc.predict(vectorizer.transform(X_test))

In [469]:
print(confusion_matrix(y_test,y_pred))

[[ 12   0   0   1   0   0   0   0   0   5   0   0   0   0   0   0   0]
 [  0   2   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0]
 [  0   0  23   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   3   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   3   1   0   0   0   0   0   0   0   0   0   0   0]
 [  1   0   3   0   0  13   1   1   0   1   0   0   0   0   0   0   0]
 [  0   0   0   0   0   3   3   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  11   0   1   0   0   0   1   0   0   0]
 [  1   0   0   0   0   0   0   0   2   0   0   0   0   0   0   0   0]
 [  6   0   2   1   0   4   1   0   0   9   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   3   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   3   0   0   0  13   0   3   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   2   2   0   0   0   0]
 [  1   0   0   0   0   1   0   1   0   0   0   2   0  41   0   0   0]
 [  0 

In [470]:
print(classification_report(y_test,y_pred))

                                   precision    recall  f1-score   support

           arm/disarm/false_alarm       0.55      0.67      0.60        18
                comm_falure_alert       1.00      0.67      0.80         3
                entry_delay_alert       0.82      1.00      0.90        23
         entry_exit_delay_setting       0.60      1.00      0.75         3
             keypad_cont_blinking       1.00      0.75      0.86         4
   keypad_pair/reset/troubleshoot       0.59      0.65      0.62        20
low_bat_alert/new_bat_not_working       0.60      0.50      0.55         6
              manage_keypad_codes       0.69      0.85      0.76        13
               new_keypad_enquiry       1.00      0.67      0.80         3
                           others       0.50      0.39      0.44        23
             panic_button_feature       1.00      1.00      1.00         3
   reset_central_station_passcode       0.76      0.65      0.70        20
          reset_home_app

# Predicting on the test data

In [471]:
test_data['cleaned_full_text']
test_data['Subcluster']=lsvc.predict(vectorizer.transform(test_data['cleaned_full_text']))

C:\Users\Gireesh.Muppalla\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [472]:
training_data=training_data.drop(['Issue Title','cleaned_text','full_chat2','cleaned_full_text'],axis=1)

In [473]:
test_data=test_data.drop(['Issue Title','cleaned_text','full_chat2','cleaned_full_text'],axis=1)

In [474]:
writer = pd.ExcelWriter('XH_Drivers_Final.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
training_data.to_excel(writer, sheet_name='Training_Data')
test_data.to_excel(writer, sheet_name='Test_Data')

In [475]:
writer.save()

In [351]:
import os
print(os.getcwd())

C:\Users\Gireesh.Muppalla\Downloads


In [137]:
#test_data.to_csv('predicted_test.csv')

# Trail and Error

In [77]:
X_train,X_test,y_train,y_test=train_test_split(training_data['cleaned_full_text'], training_data['Subcluster'],stratify=training_data['Subcluster'],random_state=5,test_size=0.20)

In [78]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(1063, 2429)

In [79]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(1063, 2429)

In [80]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [81]:
y_pred=clf.predict(tfidf_transformer.transform(count_vect.transform(X_test)))
print(classification_report(y_test,y_pred))

                                   precision    recall  f1-score   support

           arm/disarm/false_alarm       1.00      0.07      0.12        15
                comm_falure_alert       0.00      0.00      0.00         3
                entry_delay_alert       0.73      1.00      0.84        19
         entry_exit_delay_setting       0.00      0.00      0.00         2
             keypad_cont_blinking       0.00      0.00      0.00         3
   keypad_pair/reset/troubleshoot       0.27      0.25      0.26        16
low_bat_alert/new_bat_not_working       0.00      0.00      0.00         4
              manage_keypad_codes       1.00      0.10      0.18        10
               new_keypad_enquiry       0.00      0.00      0.00         2
                           others       0.33      0.17      0.22        18
             panic_button_feature       0.00      0.00      0.00         2
   reset_central_station_passcode       1.00      0.19      0.32        16
          reset_home_app

C:\Users\Gireesh.Muppalla\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [82]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=5, random_state=42))])
text_clf = text_clf.fit(X_train, y_train)

C:\Users\Gireesh.Muppalla\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [83]:
y_pred=text_clf.predict(X_test)
print(classification_report(y_test,y_pred))

                                   precision    recall  f1-score   support

           arm/disarm/false_alarm       0.50      0.73      0.59        15
                comm_falure_alert       1.00      0.33      0.50         3
                entry_delay_alert       0.73      1.00      0.84        19
         entry_exit_delay_setting       0.50      1.00      0.67         2
             keypad_cont_blinking       1.00      0.33      0.50         3
   keypad_pair/reset/troubleshoot       0.47      0.50      0.48        16
low_bat_alert/new_bat_not_working       0.40      0.50      0.44         4
              manage_keypad_codes       1.00      0.70      0.82        10
               new_keypad_enquiry       1.00      0.50      0.67         2
                           others       0.57      0.22      0.32        18
             panic_button_feature       1.00      1.00      1.00         2
   reset_central_station_passcode       0.67      0.75      0.71        16
          reset_home_app

C:\Users\Gireesh.Muppalla\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [84]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                     ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, max_iter=5, random_state=42)),
])
text_clf_svm.fit(twenty_train.data, twenty_train.target)
# predicted_svm = text_clf_svm.predict(twenty_test.data)
# np.mean(predicted_svm == twenty_test.target)

C:\Users\Gireesh.Muppalla\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf-svm',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42))])

In [85]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3),(2,2)],
               'tfidf__use_idf': (True, False),
               'clf__alpha': (1e-2, 1e-3)}

In [86]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)
y_pred=gs_clf.predict(X_test)
print(classification_report(y_test,y_pred))

                                   precision    recall  f1-score   support

           arm/disarm/false_alarm       0.53      0.67      0.59        15
                comm_falure_alert       1.00      0.33      0.50         3
                entry_delay_alert       0.68      1.00      0.81        19
         entry_exit_delay_setting       0.33      0.50      0.40         2
             keypad_cont_blinking       1.00      0.67      0.80         3
   keypad_pair/reset/troubleshoot       0.48      0.69      0.56        16
low_bat_alert/new_bat_not_working       1.00      0.25      0.40         4
              manage_keypad_codes       1.00      0.70      0.82        10
               new_keypad_enquiry       1.00      0.50      0.67         2
                           others       0.67      0.33      0.44        18
             panic_button_feature       1.00      1.00      1.00         2
   reset_central_station_passcode       0.67      0.62      0.65        16
          reset_home_app

C:\Users\Gireesh.Muppalla\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [75]:
print(gs_clf.best_score_)
print(gs_clf.best_params_)

0.8393618090452261
{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}


In [88]:
>>> from sklearn.model_selection import GridSearchCV
>>> parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
...               'tfidf__use_idf': (True, False),
...               'clf-svm__alpha': (1e-2, 1e-3),
... }
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)
gs_clf_svm.best_score_
gs_clf_svm.best_params_

C:\Users\Gireesh.Muppalla\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}